In [ ]:
! pip install langchain==0.1.2 openai

In [1]:
# 加载环境
from dotenv import load_dotenv

load_dotenv()

True

## 加载文档

In [1]:
from langchain.document_loaders import TextLoader

loader = TextLoader('./README.md', encoding='utf-8')
docs = loader.load()
docs

[Document(page_content='# LangChain 101: 05. 数据连接\n\n`LangChain`在今年的1月8号发布了v0.1.0版本。之前也断断续续的学习了一遍，奈何学了忘，忘了学。今天开始重新整理一遍，顺便记录下来，顺手写一个【LangChain极简入门课】，供小白使用（大佬可以跳过）。\n本教程默认以下前提：\n- 使用Python版本的LangChain\n- LLM使用OpenAI的gpt-3.5-turbo-1106\n- LangChain发展非常迅速，虽然已经大版本v0.1了，后续版本肯定会继续迭代，为避免教程中代码失效。本教程统一使用版本 **0.1.2**\n\n根据Langchain的[代码约定](https://github.com/langchain-ai/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。\n\n所有代码和教程开源在github：[https://github.com/keepwonder/langchain101](https://github.com/keepwonder/langchain101)\n\n----\n![](./retreival.jpg)\n\n## 什么是数据连接\n\n许多LLM应用程序需要用户特定的数据，而这些数据不是模型训练集的一部分。实现这一点的主要方法是通过检索增强生成(RAG)。在此过程中，外部数据被检索，然后传递给 LLM用来执行生成任务。\n\n`LangChain` 的数据连接概念，通过提供以下组件，实现用户数据的加载、转换、存储和查询：\n\n- 文档加载器：从不同的数据源加载文档\n- 文档转换器：拆分文档，将文档转换为问答格式，去除冗余文档，等等\n- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量\n- 向量存储：存储和搜索嵌入数据（向量）\n- 检索器：提供数据查询的通用接口\n\n我们通过实践，来看一下这些组件是如何使用的。\n\n## 数据连接实践\n\n在LLM应用连接用户数据时，通常我们会以如下步骤完成：\n1. 加载文档\n2. 拆分文档\n3. 向量化文档分块\n4. 向量数据存储\n\n这样，我们就可以通

In [2]:
docs[0].page_content

'# LangChain 101: 05. 数据连接\n\n`LangChain`在今年的1月8号发布了v0.1.0版本。之前也断断续续的学习了一遍，奈何学了忘，忘了学。今天开始重新整理一遍，顺便记录下来，顺手写一个【LangChain极简入门课】，供小白使用（大佬可以跳过）。\n本教程默认以下前提：\n- 使用Python版本的LangChain\n- LLM使用OpenAI的gpt-3.5-turbo-1106\n- LangChain发展非常迅速，虽然已经大版本v0.1了，后续版本肯定会继续迭代，为避免教程中代码失效。本教程统一使用版本 **0.1.2**\n\n根据Langchain的[代码约定](https://github.com/langchain-ai/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。\n\n所有代码和教程开源在github：[https://github.com/keepwonder/langchain101](https://github.com/keepwonder/langchain101)\n\n----\n![](./retreival.jpg)\n\n## 什么是数据连接\n\n许多LLM应用程序需要用户特定的数据，而这些数据不是模型训练集的一部分。实现这一点的主要方法是通过检索增强生成(RAG)。在此过程中，外部数据被检索，然后传递给 LLM用来执行生成任务。\n\n`LangChain` 的数据连接概念，通过提供以下组件，实现用户数据的加载、转换、存储和查询：\n\n- 文档加载器：从不同的数据源加载文档\n- 文档转换器：拆分文档，将文档转换为问答格式，去除冗余文档，等等\n- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量\n- 向量存储：存储和搜索嵌入数据（向量）\n- 检索器：提供数据查询的通用接口\n\n我们通过实践，来看一下这些组件是如何使用的。\n\n## 数据连接实践\n\n在LLM应用连接用户数据时，通常我们会以如下步骤完成：\n1. 加载文档\n2. 拆分文档\n3. 向量化文档分块\n4. 向量数据存储\n\n这样，我们就可以通过向量数据的检索器，来查询用户数据。接下来我们

In [3]:
docs[0].metadata

{'source': './README.md'}

## 拆分文档

### 按字符拆分

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

split_docs = text_splitter.split_documents(docs)

print(len(docs[0].page_content))

for split_doc in split_docs:
  print(len(split_doc.page_content))

1833
928
922
246


### 拆分代码

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

PYTHON_CODE = """
def hello_langchain():
    print("Hello, Langchain!")

# Call the function
hello_langchain()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=50, 
    chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_langchain():'),
 Document(page_content='print("Hello, Langchain!")'),
 Document(page_content='# Call the function\nhello_langchain()')]

### Markdown 文档拆分

In [6]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

markdown_document = "# Chapter 1\n\n    ## Section 1\n\nHi this is the 1st section\n\nWelcome\n\n ### Module 1 \n\n Hi this is the first module \n\n ## Section 2\n\n Hi this is the 2nd section"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = splitter.split_text(markdown_document)

splits

[Document(page_content='Hi this is the 1st section  \nWelcome', metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 1'}),
 Document(page_content='Hi this is the first module', metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 1', 'Header 3': 'Module 1'}),
 Document(page_content='Hi this is the 2nd section', metadata={'Header 1': 'Chapter 1', 'Header 2': 'Section 2'})]

### 按字符递归拆分

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

texts = text_splitter.split_documents(docs)

print(len(docs[0].page_content))

for split_doc in texts:
  print(len(split_doc.page_content))

1833
25
99
30
66
73
99
33
15
99
26
37
93
48
88
38
33
75
72
8
90
36
75
49
69
59
93
88
43
45
73
50
88


### 按token拆分

In [8]:
! pip install -q tiktoken


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=0
)

split_docs = text_splitter.split_documents(docs)

split_docs

Created a chunk of size 357, which is longer than the specified 100
Created a chunk of size 177, which is longer than the specified 100
Created a chunk of size 290, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 237, which is longer than the specified 100
Created a chunk of size 209, which is longer than the specified 100
Created a chunk of size 194, which is longer than the specified 100


[Document(page_content='# LangChain 101: 05. 数据连接', metadata={'source': './README.md'}),
 Document(page_content='`LangChain`在今年的1月8号发布了v0.1.0版本。之前也断断续续的学习了一遍，奈何学了忘，忘了学。今天开始重新整理一遍，顺便记录下来，顺手写一个【LangChain极简入门课】，供小白使用（大佬可以跳过）。\n本教程默认以下前提：\n- 使用Python版本的LangChain\n- LLM使用OpenAI的gpt-3.5-turbo-1106\n- LangChain发展非常迅速，虽然已经大版本v0.1了，后续版本肯定会继续迭代，为避免教程中代码失效。本教程统一使用版本 **0.1.2**', metadata={'source': './README.md'}),
 Document(page_content='根据Langchain的[代码约定](https://github.com/langchain-ai/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。', metadata={'source': './README.md'}),
 Document(page_content='所有代码和教程开源在github：[https://github.com/keepwonder/langchain101](https://github.com/keepwonder/langchain101)\n\n----\n![](./retreival.jpg)\n\n## 什么是数据连接', metadata={'source': './README.md'}),
 Document(page_content='许多LLM应用程序需要用户特定的数据，而这些数据不是模型训练集的一部分。实现这一点的主要方法是通过检索增强生成(RAG)。在此过程中，外部数据被检索，然后传递给 LLM用来执行生成任务。', metadata={'source': './README.md'}),
 Document(page_content='`LangCh

## 向量化文档分块

In [10]:
from langchain.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()

embeddings = embedding_model.embed_documents(
    [
        'Hello',
        'LangChain'
    ]
)

embeddings

c:\Envs\langchain_v0.1.0\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[[-0.021835500145896868,
  -0.0071507688375238845,
  -0.028395241388790658,
  -0.024550324280522062,
  -0.023524146424885817,
  0.02886286780172664,
  -0.012281656253059827,
  -0.0028706991399425337,
  -0.008332821232816149,
  -0.005429648269250388,
  0.02938245084234356,
  -0.0031856966726379323,
  -0.015613486296155584,
  -0.0026937161725531624,
  0.012262172052018154,
  -0.0009458044786013756,
  0.03881289355348649,
  0.0057121719783225595,
  0.018730988265147665,
  -0.014002776821333322,
  -0.019757164258138622,
  0.009963014908680082,
  0.005212072673085991,
  0.009021269832654605,
  -0.008170451649038691,
  -0.005228309817728267,
  0.002472893054328047,
  -0.012398563321955144,
  0.003331829577434436,
  -0.015678434874724683,
  0.0036922907891652774,
  -0.01618502782709872,
  -0.01792563259641389,
  -0.012866187872245841,
  0.004055999057295517,
  -0.016236986317424942,
  -0.0010424145346171984,
  -0.009781161123860953,
  0.021302927017037076,
  -0.008573128552082936,
  0.0130610

## 向量数据存储检索

### 存储

In [11]:
! pip install -q chromadb


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

### 检索

In [15]:
query = '什么是数据连接？'

docs = db.similarity_search(query)
print(docs[0].page_content)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


# LangChain 101: 05. 数据连接

`LangChain`在今年的1月8号发布了v0.1.0版本。之前也断断续续的学习了一遍，奈何学了忘，忘了学。今天开始重新整理一遍，顺便记录下来，顺手写一个【LangChain极简入门课】，供小白使用（大佬可以跳过）。
本教程默认以下前提：
- 使用Python版本的LangChain
- LLM使用OpenAI的gpt-3.5-turbo-1106
- LangChain发展非常迅速，虽然已经大版本v0.1了，后续版本肯定会继续迭代，为避免教程中代码失效。本教程统一使用版本 **0.1.2**

根据Langchain的[代码约定](https://github.com/langchain-ai/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。

所有代码和教程开源在github：[https://github.com/keepwonder/langchain101](https://github.com/keepwonder/langchain101)

----
![](./retreival.jpg)

## 什么是数据连接

许多LLM应用程序需要用户特定的数据，而这些数据不是模型训练集的一部分。实现这一点的主要方法是通过检索增强生成(RAG)。在此过程中，外部数据被检索，然后传递给 LLM用来执行生成任务。

`LangChain` 的数据连接概念，通过提供以下组件，实现用户数据的加载、转换、存储和查询：

- 文档加载器：从不同的数据源加载文档
- 文档转换器：拆分文档，将文档转换为问答格式，去除冗余文档，等等
- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量
- 向量存储：存储和搜索嵌入数据（向量）
- 检索器：提供数据查询的通用接口

我们通过实践，来看一下这些组件是如何使用的。

## 数据连接实践

在LLM应用连接用户数据时，通常我们会以如下步骤完成：
1. 加载文档
2. 拆分文档
3. 向量化文档分块
4. 向量数据存储


In [17]:
docs = db.similarity_search_with_score(query)
docs

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[(Document(page_content='# LangChain 101: 05. 数据连接\n\n`LangChain`在今年的1月8号发布了v0.1.0版本。之前也断断续续的学习了一遍，奈何学了忘，忘了学。今天开始重新整理一遍，顺便记录下来，顺手写一个【LangChain极简入门课】，供小白使用（大佬可以跳过）。\n本教程默认以下前提：\n- 使用Python版本的LangChain\n- LLM使用OpenAI的gpt-3.5-turbo-1106\n- LangChain发展非常迅速，虽然已经大版本v0.1了，后续版本肯定会继续迭代，为避免教程中代码失效。本教程统一使用版本 **0.1.2**\n\n根据Langchain的[代码约定](https://github.com/langchain-ai/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。\n\n所有代码和教程开源在github：[https://github.com/keepwonder/langchain101](https://github.com/keepwonder/langchain101)\n\n----\n![](./retreival.jpg)\n\n## 什么是数据连接\n\n许多LLM应用程序需要用户特定的数据，而这些数据不是模型训练集的一部分。实现这一点的主要方法是通过检索增强生成(RAG)。在此过程中，外部数据被检索，然后传递给 LLM用来执行生成任务。\n\n`LangChain` 的数据连接概念，通过提供以下组件，实现用户数据的加载、转换、存储和查询：\n\n- 文档加载器：从不同的数据源加载文档\n- 文档转换器：拆分文档，将文档转换为问答格式，去除冗余文档，等等\n- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量\n- 向量存储：存储和搜索嵌入数据（向量）\n- 检索器：提供数据查询的通用接口\n\n我们通过实践，来看一下这些组件是如何使用的。\n\n## 数据连接实践\n\n在LLM应用连接用户数据时，通常我们会以如下步骤完成：\n1. 加载文档\n2. 拆分文档\n3. 向量化文档分块\n4. 向量数据存储', metadata=